In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 60
pd.options.display.max_rows = 500
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt

## Canasta basica regional a precios constantes

### Indice de precios

In [26]:
# col_mon = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

cpi = pd.read_csv('./../data/info/indice_precios_M.csv', index_col=0)
cpi.index = pd.to_datetime(cpi.index)
cpi = cpi.loc['2000':]

cpi.columns = pd.MultiIndex.from_product([['Indice de Precios'], cpi.columns])


### Valores de las canastas regionales, basica y total

In [27]:
CBAr = pd.read_csv('./../data/info/CBA_regional.csv')
CBAr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2019,6,30), freq='M')
CBAr = CBAr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)

CBTr = pd.read_csv('./../data/info/CBT_regional.csv')
CBTr.index = pd.date_range(start = dt.datetime(2016,4,1), end = dt.datetime(2019,6,30), freq='M')
CBTr = CBTr.drop(['Ano4', 'Mes', 'TRIMESTRE'], axis = 1)

#.stack().reset_index()
CB = pd.concat([CBAr.stack(), CBTr.stack()], axis = 1)
CB.columns = ['CBA', 'CBT']; CB.index.names = ['Mes', 'Region']

In [28]:
# # 11747 (oct-nov 2019) se tiene que convertir en 
# 11747*(2283/1906) #para junio 2019
# # 11747*(1.231485) #para 2019Q2

In [34]:
cpi_M = pd.read_csv('./../data/info/indice_precios_M.csv', index_col=0)
display(cpi_M.tail())
cpi_mes_actual = cpi_M.iloc[-1][0]

,index
2020-09-30,2903.602346
2020-10-31,3012.805954
2020-11-30,3108.001733
2020-12-31,3188.361836
2021-01-31,3270.799721


In [30]:
CB_i = pd.concat([CB.unstack(), cpi], axis = 1)
CB_defl = CB_i.iloc[:, :-1].div(CB_i.iloc[:, -1], 0) # divided by cpi index

# # Check how the fillna works...
# # fig, ax = plt.subplots(1, figsize = (7, 5))
# # CB_defl.fillna(CB_defl.mean()).plot(legend = False, ax = ax)
# # CB_defl.fillna(CB_defl.quantile(.25)).head(25).plot(legend = False, ax = ax, c = '.5')
# # CB_defl.fillna(CB_defl.quantile(.75)).head(25).plot(legend = False, ax = ax, c = '.5')

CB_defl = CB_defl.fillna(CB_defl.mean()).stack()*cpi_mes_actual
CB_defl_Q = CB_defl.groupby([pd.Grouper(level = 0,freq = 'Q'), pd.Grouper(level = 1)]).mean()

CB_defl_Q.index.names = ['Q', 'Region']
CB_defl_Q = CB_defl_Q.reset_index()
CB_defl_Q.to_csv('./../data/info/CB_Reg_defl.csv', index = False)

## Listo

In [8]:
# # Debug deflacion
# frac = 0.02
# val_list = []
# for q in ['2020-06-30', '2020-03-31']:
#     df = pd.read_csv('./../data/yr_samples/RFReg_'+str(frac)+'ARG'+q+'.csv', 
#                      usecols = ['P47T'], nrows = 50000)
#     val_list += [df.median().values[0]]
# plt.plot(np.array(val_list))

In [32]:
CB_defl_Q.tail()

,Q,Region,CBA,CBT
505,2021-03-31,Gran Buenos Aires,6886.283198,16971.379763
506,2021-03-31,Noreste,6140.866592,14174.984051
507,2021-03-31,Noroeste,5985.345188,13694.944878
508,2021-03-31,Pampeana,6837.050703,16847.784886
509,2021-03-31,Patagónica,7088.929189,19831.615890
